In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📘 Cobb-Douglas Production Function: Exploring Properties

The **Cobb-Douglas production function** is fundamental in economics for modeling how inputs like capital and labor are transformed into output. This notebook interactively explores its key features.

The standard form with **constant returns to scale (CRS)** is:

$$Y = A K^{\alpha} L^{1 - \alpha}$$

- $Y$: Total output (Real GDP)
- $A$: Total Factor Productivity (TFP) - representing technology and efficiency.
- $K$: Capital input
- $L$: Labor input
- $\alpha$: Output elasticity of capital (capital's share of income, $0 < \alpha < 1$).
- $(1-\alpha)$: Output elasticity of labor (labor's share of income).

We will visualize the relationship between capital and output, the concept of diminishing marginal returns, and the property of constant returns to scale.

# 📉 Diminishing Returns and Marginal Product

While output increases when you add more capital (holding labor constant), it does so at a *decreasing rate*. This is known as **diminishing marginal returns** to a single factor.

We can see this by looking at the **Marginal Product of Capital (MPK)**:

$$MPK = \frac{\partial Y}{\partial K} = \alpha A \left(\frac{L}{K}\right)^{1-\alpha} = \alpha \frac{Y}{K}$$

The MPK tells us the additional output gained from adding one more unit of capital. As $K$ increases (with $L$ fixed), the ratio $L/K$ falls, causing the MPK to decrease.

The plot below shows:
1.  The production function ($Y$ vs $K$) getting flatter as $K$ increases.
2.  The MPK curve sloping downwards.
3.  The Average Product of Capital ($APK = Y/K$), which represents output per unit of capital. Note that $MPK = \alpha \times APK$.

# ⚖️ Constant Returns to Scale (CRS)

A key property of this specific Cobb-Douglas form is **Constant Returns to Scale**. This means if we scale *all* inputs (both $K$ and $L$) by the same factor $\lambda$, output ($Y$) will also scale by exactly that factor $\lambda$.

$$Y' = A (\lambda K)^{\alpha} (\lambda L)^{1 - \alpha} = A \lambda^{\alpha} K^{\alpha} \lambda^{1 - \alpha} L^{1 - \alpha} = \lambda^{\alpha + 1 - \alpha} (A K^{\alpha} L^{1 - \alpha}) = \lambda Y$$

The simulation demonstrates this: when you change the scaling factor $\lambda$, it calculates the output produced using $\lambda K$ and $\lambda L$ and compares it to $\lambda$ times the original output $Y$. They should be equal, confirming CRS.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def cobb_douglas_explorer(A=1.0, K_val=100, L_val=100, alpha=0.33, scale_factor=1.0, K_max_plot=200):
    """
    Calculates and plots Cobb-Douglas output, MPK, APK vs K,
    and demonstrates Constant Returns to Scale (CRS).

    Args:
        A (float): Total Factor Productivity (TFP).
        K_val (int): Specific level of Capital input for calculation point.
        L_val (int): Specific level of Labor input (fixed for Y vs K plot).
        alpha (float): Output elasticity of capital (0 < alpha < 1).
        scale_factor (float): Factor (lambda) to scale K and L for CRS check.
        K_max_plot (int): Maximum level of Capital (K) to plot on the graph.
    """
    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)
    K_val = max(K_val, 1)
    L_val = max(L_val, 1)
    A = max(A, 0.1)
    K_max_plot = max(K_max_plot, K_val, 20) # Ensure plot range is sensible
    scale_factor = max(scale_factor, 0.1)

    # --- Calculations for the specific point (K_val, L_val) ---
    Y_at_point = A * (K_val**alpha) * (L_val**(1 - alpha))

    # --- Calculations for CRS demonstration ---
    K_scaled = K_val * scale_factor
    L_scaled = L_val * scale_factor
    Y_scaled_inputs = A * (K_scaled**alpha) * (L_scaled**(1 - alpha))
    Y_scaled_output = Y_at_point * scale_factor # What output *should* be if CRS holds

    # --- Calculations for plotting Y, MPK, APK vs K (holding L=L_val) ---
    K_plot = np.linspace(1, K_max_plot, 200) # Capital range for plot
    Y_plot = A * (K_plot**alpha) * (L_val**(1 - alpha)) # Output along the range
    MPK_plot = alpha * Y_plot / K_plot # MPK along the range
    APK_plot = Y_plot / K_plot # APK along the range

    # --- Plotting ---
    fig = plt.figure(figsize=(12, 7))
    gs = fig.add_gridspec(2, 1, height_ratios=[3, 1]) # Make top plot larger

    # Top Plot: Y, MPK, APK vs K
    ax1 = fig.add_subplot(gs[0])
    color1 = 'royalblue'
    ax1.set_xlabel("Capital (K)")
    ax1.set_ylabel("Output (Y)", color=color1)
    line1 = ax1.plot(K_plot, Y_plot, color=color1, linewidth=2.5, label=f'Output Y (L={L_val})')
    ax1.scatter(K_val, Y_at_point, color='red', s=60, zorder=5, label=f'Point (K={K_val}, Y={Y_at_point:.1f})') # Mark the specific K_val point
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.set_ylim(bottom=0)
    ax1.set_xlim(0, K_max_plot)

    ax2 = ax1.twinx()
    color2 = 'forestgreen'
    color3 = 'darkorange'
    ax2.set_ylabel("Marginal/Average Product", color=color2)
    line2 = ax2.plot(K_plot, MPK_plot, color=color2, linestyle='--', linewidth=2, label=f'MPK (at L={L_val})')
    line3 = ax2.plot(K_plot, APK_plot, color=color3, linestyle=':', linewidth=2, label=f'APK (at L={L_val})')
    ax2.tick_params(axis='y', labelcolor=color2)
    ax2.set_ylim(bottom=0)

    # Combine legends
    lines = line1 + [ax1.get_children()[1]] + line2 + line3 # Include scatter point in legend
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='center left', bbox_to_anchor=(1.1, 0.5)) # Place legend outside

    ax1.set_title(f"Cobb-Douglas: Output, MPK, APK vs K (A={A:.2f}, α={alpha:.2f}, L={L_val})")

    # Bottom Plot: CRS Demonstration Bar Chart
    ax3 = fig.add_subplot(gs[1])
    labels_crs = [f'Original Y\n(K={K_val}, L={L_val})',
                  f'Scaled Inputs Y\n(K={K_scaled:.0f}, L={L_scaled:.0f})',
                  f'Scaled Output\n({scale_factor:.1f} * Original Y)']
    values_crs = [Y_at_point, Y_scaled_inputs, Y_scaled_output]
    colors_crs = ['steelblue', 'seagreen', 'lightcoral']
    bars = ax3.bar(labels_crs, values_crs, color=colors_crs)
    ax3.bar_label(bars, fmt='%.1f', padding=3)
    ax3.set_ylabel("Output Level")
    ax3.set_title(f"Constant Returns to Scale (CRS) Check (Scale Factor λ={scale_factor:.1f})")
    ax3.grid(axis='y', linestyle='--', alpha=0.6)
    # Adjust y-limit for bar chart
    ax3.set_ylim(top=max(values_crs) * 1.15)


    fig.tight_layout()
    plt.show()

    # --- Display CRS Check Results ---
    crs_check_diff = abs(Y_scaled_inputs - Y_scaled_output)
    crs_check_threshold = 1e-6 * max(Y_scaled_inputs, Y_scaled_output, 1) # Relative tolerance
    crs_holds = crs_check_diff < crs_check_threshold

    results_md = f"""
    ### 📊 Constant Returns to Scale (CRS) Verification:

    * **Original Output (Y):** {Y_at_point:.2f} (at K={K_val}, L={L_val})
    * **Output with Scaled Inputs (λK, λL):** {Y_scaled_inputs:.2f} (using λ={scale_factor:.1f})
    * **Expected Output if CRS holds (λY):** {Y_scaled_output:.2f}
    * **CRS Holds?** {'✅ Yes' if crs_holds else f'❌ No (Difference: {crs_check_diff:.2e})'}
        * *(Note: Small numerical differences are expected)*
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'} # Allow longer descriptions
interact(cobb_douglas_explorer,
         A=FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='TFP (A):', style=style),
         K_val=IntSlider(value=100, min=10, max=500, step=10, description='Capital (K):', style=style),
         L_val=IntSlider(value=100, min=10, max=500, step=10, description='Labor (L):', style=style),
         alpha=FloatSlider(value=0.33, min=0.05, max=0.95, step=0.01, description='Capital Share (alpha α):', style=style),
         scale_factor=FloatSlider(value=2.0, min=0.5, max=3.0, step=0.1, description='Scale Factor (λ):', style=style),
         K_max_plot=IntSlider(value=200, min=50, max=1000, step=20, description='Max K for Plot:', style=style)
        );

interactive(children=(FloatSlider(value=1.0, description='TFP (A):', max=3.0, min=0.5, style=SliderStyle(descr…

In [3]:
# Cobb-Douglas Production Function — CRS Interactive Simulation

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import Markdown, display

plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['figure.figsize'] = (8, 5)

# --- Cobb-Douglas Production Function with CRS Visualization ---
def cobb_douglas_crs(A_bar=1.0, K=100, L=100, alpha=0.33):
    Y = A_bar * (K**alpha) * (L**(1 - alpha))
    
    # Scale up both inputs for CRS test
    K2, L2 = 2*K, 2*L
    Y2 = A_bar * (K2**alpha) * (L2**(1 - alpha))

    fig, ax = plt.subplots()
    bars = ax.bar(['Original Y', 'Double Inputs Y'], [Y, Y2], color=['#1f77b4', '#ff7f0e'])
    ax.set_title('Cobb-Douglas: Constant Returns to Scale')
    ax.set_ylabel('Output (Y)')
    ax.bar_label(bars, fmt='%.0f')
    ax.grid(axis='y')
    plt.tight_layout()
    plt.show()

    display(Markdown(f"""
**Output (Y):** `${Y:,.0f}`  
**Output with Doubled Inputs (2K, 2L):** `${Y2:,.0f}`

---
**Constant Returns to Scale holds** if:
\[ Y_2 = 2 \cdot Y_1 \]
With Cobb-Douglas, this always occurs **when**:
\[ \alpha + (1 - \alpha) = 1 \]
    """))

interact(
    cobb_douglas_crs,
    A_bar=FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Productivity Ā'),
    K=IntSlider(value=100, min=10, max=300, step=10, description='Capital (K)'),
    L=IntSlider(value=100, min=10, max=300, step=10, description='Labor (L)'),
    alpha=FloatSlider(value=0.33, min=0.2, max=0.5, step=0.01, description='Capital Share α')
)




<>:42: SyntaxWarning: invalid escape sequence '\['
<>:42: SyntaxWarning: invalid escape sequence '\['
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_57425/2616417955.py:42: SyntaxWarning: invalid escape sequence '\['
  A_bar=FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Productivity Ā'),


interactive(children=(FloatSlider(value=1.0, description='Productivity Ā', max=2.0, min=0.5), IntSlider(value…

<function __main__.cobb_douglas_crs(A_bar=1.0, K=100, L=100, alpha=0.33)>

# 🏁 Conclusion

This interactive exploration highlights key features of the Cobb-Douglas production function with constant returns to scale:

1.  **Productivity:** TFP ($A$) scales output proportionally. Labor ($L$) also shifts the production function up.
2.  **Diminishing Returns:** Increasing capital ($K$) while holding labor ($L$) fixed yields progressively smaller increases in output, as shown by the flattening production curve and the declining MPK.
3.  **Constant Returns to Scale:** Scaling *both* capital and labor by a factor $\lambda$ scales output by the *same* factor $\lambda$, as demonstrated in the bar chart and calculations.
4.  **Factor Importance:** The capital share parameter $\alpha$ determines how strongly output responds to changes in capital versus labor, and also the relationship between MPK and APK ($MPK = \alpha \times APK$).

Understanding these properties is crucial for analyzing economic growth, factor payments, and the impact of technological change.

# 📘 Theory Summary

The Cobb-Douglas production function:
\[
Y = \bar{A} \cdot K^\alpha \cdot L^{1 - \alpha}
\]

- **Constant Returns to Scale (CRS):**
  Scaling both inputs by λ scales output by λ  
  (i.e., doubling K and L doubles Y)

- **Diminishing Returns to Capital:**
  Holding L constant, increases in K result in smaller output gains

**Sources**:
- GrowthEcon Study Guide [Ch. 3](https://growthecon.com/StudyGuide/production.html)
- Charles I. Jones, *Macroeconomics*, Ch. 4

# 📝 Guided Student Exercise: Cobb-Douglas in Action
Apply your understanding!

1. **Suppose $A=1$, $K=100$, $L=100$, and $\alpha=0.33$.**
    - What is output $Y$?
    - What is the marginal product of capital (MPK) at this point?
    - Use the interactive plot above to check your answers.

2. **Experiment:**
    - Increase $K$ while holding $L$ fixed. What happens to $Y$, MPK, and APK?
    - Now increase both $K$ and $L$ by the same factor. Does output scale by the same factor?

3. **Challenge:**
    - Try setting $\alpha$ close to 0.5 or 0.95. How does this change the shape of the production function and the importance of capital?

---
# 🌍 Real-World Data Extension: U.S. Output, Capital, and Labor
Let's see how the Cobb-Douglas function fits real data. We'll plot U.S. Real GDP, capital stock, and employment using FRED data.

```python
import pandas as pd
import plotly.graph_objects as go
gdp_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=GDPC1'
cap_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=K100001A027NBEA'
lab_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=CE16OV'
gdp = pd.read_csv(gdp_url)
cap = pd.read_csv(cap_url)
lab = pd.read_csv(lab_url)
gdp['DATE'] = pd.to_datetime(gdp['DATE'])
cap['DATE'] = pd.to_datetime(cap['DATE'])
lab['DATE'] = pd.to_datetime(lab['DATE'])
fig = go.Figure()
fig.add_trace(go.Scatter(x=gdp['DATE'], y=gdp['GDPC1'], name='Real GDP (2012$ Billions)', yaxis='y1'))
fig.add_trace(go.Scatter(x=cap['DATE'], y=cap['K100001A027NBEA'], name='Capital Stock ($ Billions)', yaxis='y2'))
fig.add_trace(go.Scatter(x=lab['DATE'], y=lab['CE16OV'], name='Employment (Millions)', yaxis='y3'))
fig.update_layout(title='U.S. Output, Capital, and Labor', xaxis_title='Year', yaxis=dict(title='Real GDP', side='left'), yaxis2=dict(title='Capital Stock', overlaying='y', side='right'), yaxis3=dict(title='Employment', anchor='free', overlaying='y', side='right', position=1))
fig.show()
```

- How do output, capital, and labor move together over time?
- Can you spot periods of rapid productivity growth?

---
# 📚 Further Reading & Resources
- [Mankiw, N. G. (2021). *Macroeconomics* (11th Edition), Chapter 3: National Income.](https://www.macmillanlearning.com/college/us/product/Macroeconomics/p/1319243584)
- [FRED: Real GDP (GDPC1)](https://fred.stlouisfed.org/series/GDPC1)
- [FRED: Private Capital Stock (K100001A027NBEA)](https://fred.stlouisfed.org/series/K100001A027NBEA)
- [FRED: Civilian Employment (CE16OV)](https://fred.stlouisfed.org/series/CE16OV)
- [GrowthEcon Study Guide: Production](https://growthecon.com/StudyGuide/production.html)

---
# 🎨 Tips for Visual Exploration
- Use the interactive plot to try different values of $A$, $K$, $L$, and $\alpha$.
- Discuss with classmates: What real-world factors might shift $A$ (TFP) over time?
- How does the Cobb-Douglas function help us understand economic growth and income distribution?

---
# 🚀 Next Steps
- Continue to the next notebook to explore growth models, technological change, and the Solow model!

# 📝 Guided Student Exercise: Cobb-Douglas in Action
Apply your understanding!

1. **Suppose $A=1$, $K=100$, $L=100$, and $\alpha=0.33$.**
    - What is output $Y$?
    - What is the marginal product of capital (MPK) at this point?
    - Use the interactive plot above to check your answers.

2. **Experiment:**
    - Increase $K$ while holding $L$ fixed. What happens to $Y$, MPK, and APK?
    - Now increase both $K$ and $L$ by the same factor. Does output scale by the same factor?

3. **Challenge:**
    - Try setting $\alpha$ close to 0.5 or 0.95. How does this change the shape of the production function and the importance of capital?

---
# 🌍 Real-World Data Extension: U.S. Output, Capital, and Labor
Let's see how the Cobb-Douglas function fits real data. We'll plot U.S. Real GDP, capital stock, and employment using FRED data.

```python
import pandas as pd
import plotly.graph_objects as go
gdp_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=GDPC1'
cap_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=K100001A027NBEA'
lab_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=CE16OV'
gdp = pd.read_csv(gdp_url)
cap = pd.read_csv(cap_url)
lab = pd.read_csv(lab_url)
gdp['DATE'] = pd.to_datetime(gdp['DATE'])
cap['DATE'] = pd.to_datetime(cap['DATE'])
lab['DATE'] = pd.to_datetime(lab['DATE'])
fig = go.Figure()
fig.add_trace(go.Scatter(x=gdp['DATE'], y=gdp['GDPC1'], name='Real GDP (2012$ Billions)', yaxis='y1'))
fig.add_trace(go.Scatter(x=cap['DATE'], y=cap['K100001A027NBEA'], name='Capital Stock ($ Billions)', yaxis='y2'))
fig.add_trace(go.Scatter(x=lab['DATE'], y=lab['CE16OV'], name='Employment (Millions)', yaxis='y3'))
fig.update_layout(title='U.S. Output, Capital, and Labor', xaxis_title='Year', yaxis=dict(title='Real GDP', side='left'), yaxis2=dict(title='Capital Stock', overlaying='y', side='right'), yaxis3=dict(title='Employment', anchor='free', overlaying='y', side='right', position=1))
fig.show()
```

- How do output, capital, and labor move together over time?
- Can you spot periods of rapid productivity growth?

---
# 📚 Further Reading & Resources
- [Mankiw, N. G. (2021). *Macroeconomics* (11th Edition), Chapter 3: National Income.](https://www.macmillanlearning.com/college/us/product/Macroeconomics/p/1319243584)
- [FRED: Real GDP (GDPC1)](https://fred.stlouisfed.org/series/GDPC1)
- [FRED: Private Capital Stock (K100001A027NBEA)](https://fred.stlouisfed.org/series/K100001A027NBEA)
- [FRED: Civilian Employment (CE16OV)](https://fred.stlouisfed.org/series/CE16OV)
- [GrowthEcon Study Guide: Production](https://growthecon.com/StudyGuide/production.html)

---
# 🎨 Tips for Visual Exploration
- Use the interactive plot to try different values of $A$, $K$, $L$, and $\alpha$.
- Discuss with classmates: What real-world factors might shift $A$ (TFP) over time?
- How does the Cobb-Douglas function help us understand economic growth and income distribution?

---
# 🚀 Next Steps
- Continue to the next notebook to explore growth models, technological change, and the Solow model!